<a href="https://colab.research.google.com/github/sm2hashmi/Syed-Muhammad-Masab-Hashmi/blob/master/Deep%20Learning%20Project%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy
import pandas
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline


In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)


In [20]:
# load dataset
from google.colab import files
uploaded = files.upload()
import io
dataframe = pandas.read_csv(io.BytesIO(uploaded['iris.csv']))
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]


Saving iris.csv to iris (1).csv


In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


In [0]:
def create_model():
   model = Sequential()
   model.add(layers.Dense(4, activation= 'softmax', input_shape = (4,)))
   model.add(layers.Dense(3, activation = 'relu'))
    
   model.compile(optimizer='Adam', loss ='categorical_crossentropy', metric = ['accuracy'])
  
   return model

In [0]:
estimator = KerasClassifier(build_fn=create_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: ignored